MUKESH P
212222240068


In [2]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk


  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-plmz5mnw/gym-walk_c558de2faffd4e3b8d2525aa5834f2b4
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-plmz5mnw/gym-walk_c558de2faffd4e3b8d2525aa5834f2b4
  Resolved https://github.com/mimoralea/gym-walk to commit b915b94cf2ad16f8833a1ad92ea94e88159279f5
  Preparing metadata (setup.py) ... done
  Created wheel for gym-walk: filename=gym_walk-0.0.2-py3-none-any.whl size=5377 sha256=b33c3b21b0fc90aa11f58f425fc6d8e3e0e9da8a0bb22b8547d35395a6389825
  Stored in directory: /tmp/pip-ephem-wheel-cache-l18ehzs7/wheels/bf/23/e5/a94be4a90dd18f7ce958c21f192276cb01ef0daaf2bc66583b
Successfully built gym-walk


In [3]:
import warnings ; warnings.filterwarnings('ignore')


In [4]:
import gym
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123);

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [5]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")


In [6]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [7]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, h = env.step(pi(state))
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)


In [8]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi(state))
            results[-1] += reward
            steps += 1
    return np.mean(results)

In [9]:
env = gym.make('FrozenLake-v1')
P = env.env.P
init_state = env.reset()
goal_state = 15
LEFT, DOWN, RIGHT, UP = range(4)

In [10]:
P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [11]:
init_state

0

In [12]:
pi_frozenlake = lambda s: {
    0: RIGHT,
    1: DOWN,
    2: RIGHT,
    3: LEFT,
    4: DOWN,
    5: LEFT,
    6: RIGHT,
    7:LEFT,
    8: UP,
    9: DOWN,
    10:LEFT,
    11:DOWN,
    12:RIGHT,
    13:RIGHT,
    14:DOWN,
    15:LEFT #Stop
}[s]
print_policy(pi_frozenlake, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)

Policy:
| 00      > | 01      v | 02      > | 03      < |
| 04      v |           | 06      > |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |


In [ ]:
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_frozenlake, goal_state=goal_state) * 100,
    mean_return(env, pi_frozenlake)))

Reaches goal 10.00%. Obtains an average undiscounted return of 0.1000.


In [13]:
pi_2 = lambda s: {
    0: DOWN,
    1: DOWN,
    2: DOWN,
    3: LEFT,
    4: RIGHT,
    5: DOWN,
    6: DOWN,
    7: LEFT,
    8: RIGHT,
    9: DOWN,
    10: DOWN,
    11: DOWN,
    12: RIGHT,
    13: RIGHT,
    14: DOWN,
    15: LEFT  # Stop
}[s]

In [14]:
print("Name: MUKESH P")
print("Register Number: 212222240068")
print_policy(pi_2, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)

Name: MUKESH P
Register Number: 212222240068
Policy:
| 00      v | 01      v | 02      v | 03      < |
| 04      > |           | 06      v |           |
| 08      > | 09      v | 10      v |           |
|           | 13      > | 14      v |           |


In [ ]:
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_2, goal_state=goal_state) * 100,
    mean_return(env, pi_2)))


Reaches goal 7.00%. Obtains an average undiscounted return of 0.0700.


In [ ]:
# Compare the two policies
success1 = probability_success(env, pi_frozenlake, goal_state=goal_state)
success2 = probability_success(env, pi_2, goal_state=goal_state)

mean1 = mean_return(env, pi_frozenlake)
mean2 = mean_return(env, pi_2)

print("\n--- Policy Comparison ---")
print("First Policy -> Success: {:.2f}%, Mean Return: {:.4f}".format(success1*100, mean1))
print("Second Policy -> Success: {:.2f}%, Mean Return: {:.4f}".format(success2*100, mean2))

if success1 > success2:
    print("The first policy performs better")
elif success2 > success1:
    print("The second policy performs better")
else:
    print("Both policies perform equally well (in terms of success)")



--- Policy Comparison ---
First Policy -> Success: 10.00%, Mean Return: 0.1000
Second Policy -> Success: 7.00%, Mean Return: 0.0700
The first policy performs better


In [ ]:
def policy_evaluation(pi, P, gamma=1.0, theta=1e-10):
    V = np.zeros(len(P), dtype=np.float64)
    while True:
        delta = 0
        for s in range(len(P)):
            v = 0
            a = pi(s)  # action chosen by the policy at state s
            for prob, next_state, reward, done in P[s][a]:
                v += prob * (reward + gamma * V[next_state])
            delta = max(delta, abs(v - V[s]))
            V[s] = v
        if delta < theta:
            break
    return V


In [ ]:
# Code to evaluate the first policy
V1 = policy_evaluation(pi_frozenlake, P,gamma=0.99)
print_state_value_function(V1, P, n_cols=4, prec=5)

State-value function:
| 00 0.11448 | 01 0.08191 | 02 0.13372 | 03 0.06586 |
| 04 0.15053 |           | 06 0.20562 |           |
| 08 0.30562 | 09 0.46997 | 10 0.48938 |           |
|           | 13 0.62915 | 14 0.80739 |           |


In [ ]:
# Code to evaluate the second policy
V2 = policy_evaluation(pi_2, P, gamma=0.99)
print_state_value_function(V2, P, n_cols=4, prec=5)

# Compare the two policies based on the value function
if np.all(V1 >= V2):
    print("The first policy is better")
elif np.all(V2 >= V1):
    print("The second policy is better")
else:
    print("Both policies have their merits")


State-value function:
| 00 0.04728 | 01 0.03608 | 02 0.06206 | 03 0.03057 |
| 04 0.05991 |           | 06 0.12141 |           |
| 08 0.13426 | 09 0.34694 | 10 0.36792 |           |
|           | 13 0.54914 | 14 0.76798 |           |
The first policy is better


In [ ]:
if(np.sum(V1>=V2)):
  print("The first policy is the better policy")
elif(np.sum(V2>=V1)):
  print("The second policy is the better policy")
else:
  print("Both policies have their merits.")

The first policy is the better policy


MUKESH P
212222240068
